# EasyVVUQ - Basic Concepts

This tutorial shows a simple EasyVVUQ workflow in action. The example used here is a simulation of a robotic arm. Description of it can be found [here](https://www.sfu.ca/~ssurjano/robot.html).

The usage of the application is:

```robot <input_file>```

It outputs a single file called output.json, which will look like ```{'distance': 30}``` or similar where ```distance``` is the distance between the plane to which the robot arm is attached and the end of the arm.

The robot.template is a template input file, in JSON format

```{"outfile": "$out_file", "theta1": $theta1, "theta2" $theta2, "theta3": $theta3, "theta4": $theta4, "L1": $L1, "L2": $L2, "L3": $L3, "L4": $L4}```

The thetas are angles (in radians) between the four robotic arm segments and L's are their lengths (in arbitrary units). The values for each key are tags (signified by the $ delimiter) which will be substituted by EasyVVUQ with values to sample the parameter space. In the following tutorial, the template will be used to generate files called input.json that will be the input to each run of ```robot```.

So, for example:

In [ ]:
!echo "{\"outfile\": \"output.json\", \"theta1\": 0.5, \"theta2\": 0.5, \"theta3\": 0.25, \"theta4\": 0.75, \"L1\": 1, \"L2\": 1, \"L3\": 1, \"L4\": 1}" > input.json

In [ ]:
!./robot input.json

In [ ]:
!cat output.json

# Campaign

In [1]:
import easyvvuq as uq
import chaospy as cp

In [2]:
params = {
    "theta1": {"type": "float", "min": 0.0, "max": 1.0, "default": 0.5}, 
    "theta2": {"type": "float", "min": 0.0, "max": 1.0, "default": 0.5}, 
    "theta3": {"type": "float", "min": 0.0, "max": 1.0, "default": 0.5}, 
    "theta4": {"type": "float", "min": 0.0, "max": 1.0, "default": 0.5},
    "L1": {"type": "float", "min": 0.0, "default": 1.0},
    "L2": {"type": "float", "min": 0.0, "default": 1.0},
    "L3": {"type": "float", "min": 0.0, "default": 1.0},
    "L4": {"type": "float", "min": 0.0, "default": 1.0},
    "outfile": {"type": "string", "default": "output.json"}
}

In [3]:
encoder = uq.encoders.GenericEncoder(template_fname='robot.template', delimiter='$', target_filename='input.json')
decoder = uq.decoders.JSONDecoder(target_filename='output.json', output_columns=['distance'])

In [4]:
campaign = uq.Campaign(name='robot', params=params, encoder=encoder, decoder=decoder)

In [5]:
vary = {
    "theta1": cp.Uniform(0, 1),
    "theta2": cp.Uniform(0, 1),
    "theta3": cp.Uniform(0, 1),
    "theta4": cp.Uniform(0, 1)
}

In [6]:
campaign.set_sampler(uq.sampling.PCESampler(vary=vary, polynomial_order=3))

In [14]:
execution = campaign.sample_and_apply(action=uq.actions.ExecuteLocalV2("robot input.json"))

In [15]:
execution.start();

In [17]:
execution.progress()

{'ready': 0, 'active': 0, 'finished': 0, 'failed': 0}

In [18]:
results = campaign.analyse(qoi_cols=['distance'])

In [19]:
results.sobols_first()

{'distance': {'theta1': array([1.89233086e-29]),
  'theta2': array([0.27726841]),
  'theta3': array([0.28837432]),
  'theta4': array([0.27726841])}}

Alternatively you can try using Stochastic Collocation. The procedure is exactly the same.

In [20]:
campaign.set_sampler(uq.sampling.SCSampler(vary=vary, polynomial_order=3))

Received integer polynomial order, assuming isotropic grid


In [21]:
execution = campaign.sample_and_apply(action=uq.actions.ExecuteLocalV2("robot input.json"))

In [22]:
execution.start();

In [29]:
execution.progress()

{'ready': 0, 'active': 0, 'finished': 256, 'failed': 0}

In [30]:
results = campaign.analyse(qoi_cols=['distance'])

Loading samples...
done
Computing Sobol indices...
done.


In [33]:
results.sobols_first()

{'distance': {'theta1': array([1.95649716e-15]),
  'theta2': array([0.13958238]),
  'theta3': array([0.14517331]),
  'theta4': array([0.13958238])}}

In [32]:
campaign.collate()

,run_id,theta1,theta2,theta3,theta4,L1,L2,L3,L4,outfile,distance
,0,0,0,0,0,0,0,0,0,0,0
0,257,0.069432,0.069432,0.069432,0.069432,1.0,1.0,1.0,1.0,output.json,3.539460
1,258,0.069432,0.069432,0.069432,0.330009,1.0,1.0,1.0,1.0,output.json,2.090885
2,259,0.069432,0.069432,0.069432,0.669991,1.0,1.0,1.0,1.0,output.json,2.921915
3,260,0.069432,0.069432,0.069432,0.930568,1.0,1.0,1.0,1.0,output.json,3.812682
4,261,0.069432,0.069432,0.330009,0.069432,1.0,1.0,1.0,1.0,output.json,1.213296
...,...,...,...,...,...,...,...,...,...,...,...
251,508,0.930568,0.930568,0.669991,0.930568,1.0,1.0,1.0,1.0,output.json,1.213296
252,509,0.930568,0.930568,0.930568,0.069432,1.0,1.0,1.0,1.0,output.json,3.812682
253,510,0.930568,0.930568,0.930568,0.330009,1.0,1.0,1.0,1.0,output.json,2.921915
